In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 15
pd.options.display.max_columns = 300

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from plotly.subplots import make_subplots

from plotly import graph_objects as go

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torchmetrics
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
# Data = pd.read_csv( '../input/dmspparticleprecipitatefluxprediction/AI_Ready_DMSP_Data.csv', parse_dates=['Datetimes'] )
Data = pd.read_csv('../input/dmspparticleprecipitatefluxprediction/Compressed_Data/Compressed_Data.csv')
Data.drop(columns=['Unnamed: 0'],inplace=True)

In [3]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1945887 entries, 0 to 1945886
Columns: 152 entries, SC_AACGM_LAT to newell_5min
dtypes: float64(152)
memory usage: 2.2 GB


Data contain ID, Datetime and Numerical features (both discrete and continuos)

# Checking Null Values

In [4]:
def missing( df ):
#     high_missing_values_features = []
#     missing_values_features_less_5 = []
#     missing_values_features_btw_5_and_40 = []
    missing_values_features = []
    for column in df.columns.values:
        print( "Feature:- ", column )
        percent_missing = df[column].isnull().sum()/df[column].shape[0]*100
        print('Percentage of Missing Values:- ', percent_missing )
    
#         if percent_missing>=25:
#             high_missing_values_features.append( column )
#         elif percent_missing<=40:
#             missing_values_features_less_5.append( column )
#         elif percent_missing>5 & percent_missing<=40:
#             missing_values_features_btw_5_and_25.append( column )
        print("--------------------------------------------------------------")
        print("\n")
    return # missing_values_features_less_5, missing_values_features_btw_5_and_40,  high_missing_values_features

missing(Data)

Feature:-  SC_AACGM_LAT
Percentage of Missing Values:-  0.0
--------------------------------------------------------------


Feature:-  SC_AACGM_LTIME
Percentage of Missing Values:-  0.0
--------------------------------------------------------------


Feature:-  ELE_TOTAL_ENERGY_FLUX
Percentage of Missing Values:-  0.0
--------------------------------------------------------------


Feature:-  ELE_TOTAL_ENERGY_FLUX_STD
Percentage of Missing Values:-  0.0
--------------------------------------------------------------


Feature:-  ELE_AVG_ENERGY
Percentage of Missing Values:-  0.0
--------------------------------------------------------------


Feature:-  ELE_AVG_ENERGY_STD
Percentage of Missing Values:-  0.0
--------------------------------------------------------------


Feature:-  sin_ut
Percentage of Missing Values:-  0.0
--------------------------------------------------------------


Feature:-  cos_ut
Percentage of Missing Values:-  0.0
---------------------------------------------

In [5]:
Data

,SC_AACGM_LAT,SC_AACGM_LTIME,ELE_TOTAL_ENERGY_FLUX,ELE_TOTAL_ENERGY_FLUX_STD,ELE_AVG_ENERGY,ELE_AVG_ENERGY_STD,sin_ut,cos_ut,sin_doy,cos_doy,sin_SC_AACGM_LTIME,cos_SC_AACGM_LTIME,F107,Bz,By,Bx,AE,AL,AU,SymH,PC,vsw,vx,psw,borovsky,newell,F107_6hr,Bz_6hr,By_6hr,Bx_6hr,AE_6hr,AL_6hr,AU_6hr,SymH_6hr,PC_6hr,vsw_6hr,vx_6hr,psw_6hr,borovsky_6hr,newell_6hr,F107_5hr,Bz_5hr,By_5hr,Bx_5hr,AE_5hr,AL_5hr,AU_5hr,SymH_5hr,PC_5hr,vsw_5hr,vx_5hr,psw_5hr,borovsky_5hr,newell_5hr,F107_3hr,Bz_3hr,By_3hr,Bx_3hr,AE_3hr,AL_3hr,AU_3hr,SymH_3hr,PC_3hr,vsw_3hr,vx_3hr,psw_3hr,borovsky_3hr,newell_3hr,F107_1hr,Bz_1hr,By_1hr,Bx_1hr,AE_1hr,AL_1hr,AU_1hr,SymH_1hr,PC_1hr,vsw_1hr,vx_1hr,psw_1hr,borovsky_1hr,newell_1hr,F107_45min,Bz_45min,By_45min,Bx_45min,AE_45min,AL_45min,AU_45min,SymH_45min,PC_45min,vsw_45min,vx_45min,psw_45min,borovsky_45min,newell_45min,F107_30min,Bz_30min,By_30min,Bx_30min,AE_30min,AL_30min,AU_30min,SymH_30min,PC_30min,vsw_30min,vx_30min,psw_30min,borovsky_30min,newell_30min,F107_15min,Bz_15min,By_15min,Bx_15min,AE_15min,AL_15min,AU_15min,SymH_15min,PC_15min,vsw_15min,vx_15min,psw_15min,borovsky_15min,newell_15min,F107_10min,Bz_10min,By_10min,Bx_10min,AE_10min,AL_10min,AU_10min,SymH_10min,PC_10min,vsw_10min,vx_10min,psw_10min,borovsky_10min,newell_10min,F107_5min,Bz_5min,By_5min,Bx_5min,AE_5min,AL_5min,AU_5min,SymH_5min,PC_5min,vsw_5min,vx_5min,psw_5min,borovsky_5min,newell_5min
0,51.16,5.664,2.057827e+08,0.41110,39.90,0.4114,-0.2461,-0.9690,0.2051,0.9785,0.9960,0.0878,68.9,2.78,6.40,-3.49,82.0,-58.0,24.0,3.0,0.77,354.8,-354.0,3.20,14199.369,1848.0,68.9,3.143,5.215,-2.877,93.3,-18.75,74.30,-7.9180,1.342,337.8,-337.2,3.977,13556.319,1206.0,68.9,1.15500,6.504,0.43750,52.40,-4.25,47.9,3.000,0.9610,346.2,-345.8,4.914,29030.967,2644.0,68.9,3.436,9.125,0.3650,79.50,-48.66,30.0,7.668,0.270,362.8,-361.5,5.330,19770.271,2784.0,68.9,3.980,4.960,0.578,214.9,-169.5,45.0,7.500,0.830,368.2,-368.0,4.766,15725.455,1005.5,68.9,2.81,6.06,1.00,137.0,-107.0,30.0,7.0,0.53,363.8,-363.5,4.42,19981.770,1765.0,68.9,0.56,7.67,-2.12,74.0,-48.0,25.0,5.0,0.51,354.0,-353.8,3.61,23353.9060,3500.0,68.9,1.72,7.02,-2.64,61.0,-37.0,23.0,3.0,0.61,354.5,-354.0,3.13,17383.697,2592.0,68.9,1.25,6.52,-3.03,56.0,-36.0,19.0,4.0,0.71,364.5,-363.5,3.09,19335.344,2762.0,68.9,1.51,6.42,-2.72,86.0,-61.0,25.0,3.0,0.77,356.0,-355.5,2.96,17439.640,2490.0
1,54.12,5.535,2.509837e+08,0.47830,40.80,0.4785,-0.2505,-0.9683,0.2051,0.9785,0.9927,0.1212,68.9,2.78,6.40,-3.49,82.0,-58.0,24.0,3.0,0.77,354.8,-354.0,3.20,14199.369,1848.0,68.9,3.143,5.215,-2.877,93.3,-18.75,74.30,-7.9180,1.342,337.8,-337.2,3.977,13556.319,1206.0,68.9,1.15500,6.504,0.43750,52.40,-4.25,47.9,3.000,0.9610,346.2,-345.8,4.914,29030.967,2644.0,68.9,3.436,9.125,0.3650,79.50,-48.66,30.0,7.668,0.270,362.8,-361.5,5.330,19770.271,2784.0,68.9,3.980,4.960,0.578,214.9,-169.5,45.0,7.500,0.830,368.2,-368.0,4.766,15725.455,1005.5,68.9,2.81,6.06,1.00,137.0,-107.0,30.0,7.0,0.53,363.8,-363.5,4.42,19981.770,1765.0,68.9,0.56,7.67,-2.12,74.0,-48.0,25.0,5.0,0.51,354.0,-353.8,3.61,23353.9060,3500.0,68.9,1.72,7.02,-2.64,61.0,-37.0,23.0,3.0,0.61,354.5,-354.0,3.13,17383.697,2592.0,68.9,1.25,6.52,-3.03,56.0,-36.0,19.0,4.0,0.71,364.5,-363.5,3.09,19335.344,2762.0,68.9,1.51,6.42,-2.72,86.0,-61.0,25.0,3.0,0.77,356.0,-355.5,2.96,17439.640,2490.0
2,62.94,5.030,4.469146e+08,0.60350,143.80,0.6040,-0.2630,-0.9650,0.2051,0.9785,0.9680,0.2510,68.9,1.13,6.49,-3.20,96.0,-75.0,21.0,1.0,0.57,350.8,-350.5,3.28,20729.744,2684.0,68.9,2.977,5.297,-2.363,80.2,-14.50,65.50,-6.5820,1.236,338.0,-337.8,4.080,15091.975,1330.0,68.9,1.09500,6.574,0.11664,55.34,-6.50,48.6,3.416,0.9893,346.5,-346.0,4.960,29352.023,2712.0,68.9,2.738,9.200,-0.8135,85.30,-53.34,31.0,8.000,0.295,361.8,-360.5,5.280,21831.275,3088.0,68.9,3.482,5.082,0.345,194.6,-152.5,42.0,7.668,0.763,367.0,-366.5,4.770,17715.834,1181.0,68.9,4.45,5.58,2.49,127.0,-92.0,34.0,6.0,0.62,365.0,-364.5,4.11,11208.996,1043.0,68.9,1.14,7.16,-2.70,72.0,-44.0,27.0,4.0,0.52,359.0,-358.5,3.66,22319.2360,3020.0,68.9,1.25,6.52,-3.03,

Log-transforming the target


In [6]:
Data['ELE_TOTAL_ENERGY_FLUX'] = Data['ELE_TOTAL_ENERGY_FLUX'].apply(lambda x: np.log(x))

No Null Values are present

In [7]:
# Data_ID = Data[["ID_SC"]]
# Data_DateTime = Data[['Datetimes']]
# Data.drop(columns=["ID_SC"], inplace=True)
# Data.drop(columns=["Datetimes"], inplace=True)

# Train Test Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split( Data.drop(['ELE_TOTAL_ENERGY_FLUX'], axis=1), Data[['ELE_TOTAL_ENERGY_FLUX']], test_size=0.3, random_state=10)

Normalization

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)

X_train = pd.DataFrame( scaler.transform( X_train ), columns=X_train.columns )
X_test = pd.DataFrame( scaler.transform( X_test ), columns=X_test.columns )

Principle Component Analysis (PCA)

In [10]:
from sklearn.decomposition import PCA
# PCA-fitting the training set
pca = PCA(n_components = 30)
pca.fit(X_train)

# PCA summary
print(pd.Series({"Number of components": "{}".format(pca.n_components_),
                 "Explained variance ratio": "{:.4g}%".format(pca.explained_variance_ratio_.sum()*100)
                }).to_string())

Number of components            30
Explained variance ratio    96.11%


In [11]:
# PCA-transforming the training set features
X_train = pd.DataFrame(pca.transform(X_train))
print(pd.Series({"Shape": "{}".format(X_train.shape)}).to_string())
print(" ")
X_train.head()

# PCA transforming the test set features
X_test = pd.DataFrame(pca.transform(X_test))
print(pd.Series({"Shape": "{}".format(X_test.shape)}).to_string())
print(" ")
X_test.head()

Shape    (1362120, 30)
 
Shape    (583767, 30)
 


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,-0.404130,0.693339,0.664914,0.135653,0.042902,-0.496564,0.217751,-0.445488,-0.016054,-0.153167,-0.103358,0.151880,-0.199611,-0.184911,0.235978,-0.086302,-0.173886,-0.144817,0.003938,0.037464,-0.049998,-0.071542,-0.011358,0.047491,0.048742,-0.041321,-0.020536,-0.036823,-0.069954,0.007098
1,1.321985,-0.027777,0.618108,0.318163,-0.002905,-0.435467,0.157803,0.015606,0.109618,-0.363888,0.327759,-0.139473,0.070134,-0.116925,0.078236,-0.089849,-0.200809,0.073937,-0.069251,0.396685,0.104562,0.032123,0.134628,0.021127,0.109098,-0.215288,-0.035587,-0.071828,0.112441,0.159793
2,1.031090,-0.050193,-0.494048,0.800504,0.088004,0.596254,0.024706,0.477988,0.230234,-0.282252,-0.173399,0.341903,-0.140630,-0.367300,0.078661,0.261246,0.010044,0.072930,0.187174,-0.000397,0.065906,0.061644,-0.210797,-0.027873,-0.056448,0.074148,0.116225,0.292916,-0.250456,0.098026
3,-1.040452,-0.031519,-0.280293,-0.068034,0.531815,0.730497,0.089190,0.052744,-0.093702,-0.359195,0.047930,0.280276,-0.065744,-0.146228,0.154505,0.097556,0.071390,0.058021,-0.042120,-0.133098,0.035877,-0.005648,0.126824,-0.099017,-0.347668,0.060849,0.007024,-0.028510,-0.015869,0.028155
4,-0.445973,-0.587238,-0.541433,-0.866581,0.159837,-0.435885,0.037106,-0.263822,-0.317437,-0.258337,-0.649021,-0.259034,0.179563,-0.221140,0.246690,0.290433,-0.168077,-0.063686,-0.027146,-0.170317,-0.093438,0.036398,0.124373,-0.182857,-0.028717,0.045675,-0.044985,0.044795,-0.055567,-0.295825


In [12]:
# for col in Data.columns:
#     col_type = Data[col].dtype

#     if col_type != object:
#         c_min = Data[col].min()
#         c_max = Data[col].max()
#         if str(col_type)[:3] == 'int':
#             if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                 Data[col] = Data[col].astype(np.int8)
#             elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                 Data[col] = Data[col].astype(np.int16)
#             elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                 Data[col] = Data[col].astype(np.int32)
#             elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                 Data[col] = Data[col].astype(np.int64)  
#         else:
#             if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                 Data[col] = Data[col].astype(np.float16)
#             elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                 Data[col] = Data[col].astype(np.float32)
#             else:
#                 Data[col] = Data[col].astype(np.float64)
#     else:
#         Data[col] = Data[col].astype('category')

# Data.to_csv('Compressed_Data.csv')

In [13]:
# # Define a size for your train set 
# train_size = int(0.7 * len(Data))


# # Shuffle
# Data = Data.sample(frac=1)

# # Split your dataset 
# X_train_set = Data.drop(columns=['ELE_TOTAL_ENERGY_FLUX'])[:train_size]
# X_test_set = Data.drop(columns=['ELE_TOTAL_ENERGY_FLUX'])[train_size:]

# y_train_set = Data.drop(columns=['ELE_TOTAL_ENERGY_FLUX'])[:train_size]
# y_test_set = Data.drop(columns=['ELE_TOTAL_ENERGY_FLUX'])[train_size:]

# Baseline Neural Network using Pytorch

Basic Parameters

In [14]:
EPOCHS = 20
BATCH_SIZE = 64
LEARNING_RATE = 0.001

Dataset for PyTorch

In [15]:
class Train(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    def __len__ (self):
        return len(self.X_data)

class Test(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    def __len__ (self):
        return len(self.X_data)
    
########################

train_data = Train(torch.FloatTensor(X_train.values), torch.FloatTensor(y_train.values))
test_data = Test(torch.FloatTensor(X_test.values), torch.FloatTensor(y_test.values))

########################

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

Neural Network

In [16]:
class NN_Model(nn.Module):
    def __init__(self):
        super(NN_Model, self).__init__()
        # Number of input features is 129.
        self.layer_1 = nn.Linear(X_train.shape[1], 100) 
        self.layer_2 = nn.Linear(100, 75)
        self.layer_3 = nn.Linear(75, 50)
        self.layer_4 = nn.Linear(50, 25)
        self.layer_out = nn.Linear(25, 1) 
        
        self.relu = nn.ReLU()
    
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.relu(self.layer_2(x))
        x = self.relu(self.layer_3(x))
        x = self.relu(self.layer_4(x))
        x = self.layer_out(x)
        
        return x
    
    def forward(self, test_inputs):
        x = self.relu(self.layer_1(test_inputs))
        x = self.relu(self.layer_2(x))
        x = self.relu(self.layer_3(x))
        x = self.relu(self.layer_4(x))
        x = self.layer_out(x)
        
        return x
    

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

##############

model = NN_Model()
model.to(device)
print(model)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

##############

cuda:0
NN_Model(
  (layer_1): Linear(in_features=30, out_features=100, bias=True)
  (layer_2): Linear(in_features=100, out_features=75, bias=True)
  (layer_3): Linear(in_features=75, out_features=50, bias=True)
  (layer_4): Linear(in_features=50, out_features=25, bias=True)
  (layer_out): Linear(in_features=25, out_features=1, bias=True)
  (relu): ReLU()
)


Metric

In [17]:
# from torchmetrics import R2Score
# def R2_Score(y_pred, y_test):
#     # print( y_pred )
#     # print( y_test )
#     r2score = R2Score(num_outputs=1)
#     acc = r2score(y_pred, y_test)
#     return acc
from sklearn.metrics import r2_score

def R2_Score(y_pred, y_test):
    acc = r2_score(y_pred.numpy(), y_test.asarray())
    return acc

Train Neural Network

In [19]:
losss = []
epoch_losss = []
val_losss = []
val_epoch_losss = []
epoch_accc = []
accs = []
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    val_epoch_loss = 0
    val_epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch)
        # acc = R2_Score(y_pred, y_test)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        #epoch_acc += acc.item()
        losss.append(epoch_loss)
        #accs.append(epoch_acc)
        
        
     # Validation
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
        
            val_epoch_loss += loss.item()
            epoch_acc += acc.item()
            val_losss.append(epoch_loss)
            accs.append(epoch_acc)
        
        
    epoch_losss.append(epoch_loss/len(train_loader))
    epoch_accc.append(epoch_acc/len(train_loader))
    val_epoch_losss.append(val_epoch_loss/len(test_loader))
    epoch_accc.append(epoch_acc/len(train_loader))

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(test_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

NameError: name 'acc' is not defined

Plot Loss and Accuracy

In [ ]:
plt.plot(epoch_losss, label='loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(epoch_accc, label='accuracy')
plt.legend()
plt.show()

In [ ]:
# y_pred_list = []
# model.eval()
# with torch.no_grad():
#     for X_batch in test_loader:
#         X_batch = X_batch.to(device)
#         y_test_pred = model(X_batch)
#         y_test_pred = torch.sigmoid(y_test_pred)
#         y_pred_list.append(y_test_pred.cpu().numpy())

# y_pred_list = [a.squeeze().tolist() for a in y_pred_list]